In [1]:
from __future__ import annotations

import ibis

from mismo.block import KeyBlocker
from mismo.playdata import load_febrl1

ibis.options.interactive = True
records, links = load_febrl1()

In [2]:
from mismo.cluster import connected_components
from mismo.eda import string_comparator_score_chart, string_comparator_scores

In [3]:
connected = connected_components(links=links, records=records)

In [4]:
blocked = KeyBlocker("component")(connected, connected)

In [5]:
scores = string_comparator_scores(blocked.limit(20), "surname_l", "surname_r")

In [7]:
string_comparator_score_chart(blocked.limit(20), "surname_l", "surname_r")

alt.HConcatChart(...)